This notebook is created to use it for coursera's IBM data science capstone.This note book has been created on IBM watson platform.

# week 1 assignment(just creating a notebook)

In [2]:
import numpy as np
import pandas as pd


In [3]:
print("Hello capstone project!!")

Hello capstone project!!


# This is assignment from week 3

# Part -1 of the assignment

# 1. Scrap the table from wikipedea

We will scrap postal codes and neighbourhood of Toronto and make it as pandas dataframe to perform segmentation and clustering.<br>
wiki page: <a>https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</a><br>
We need to extract information from table from wikipedia page<br>


<h3>first method : Using pandas</h3>

In [4]:
toronto_ps=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",header=None) #returns all the table in form of lists from the web page
toronto_ps[0].head()    #we want the first table

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Assigning toronto_ps[0] to dataframe toronto_df because we are interested only in 1st table scraped from wikipedia.

In [5]:
toronto=toronto_ps[0]
toronto.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<h3>2nd method using beautifulSoup</h3>

In [6]:
!pip install beautifulsoup4

In [7]:
from bs4 import BeautifulSoup
import requests

In [8]:
url=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(url,'lxml')
# print(soup.prettify()) #commenting because it prints a long output

After inspecting elements on wikipedia we can see that table is conatined in the class wikitable.Hence we extract from there.

In [11]:
my_table=soup.find('table',{'class':'wikitable'})
# my_table

In [15]:
#places are mentioned in <tr> tag.
rows=my_table.findAll('tr')
print(rows[0])
rows[1] #instances from the list treturned in rows

<tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>


<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>

In [19]:
del rows[0] #they contain the table header(will insert the header later in dataframe)

In [21]:
print(rows[0].findAll('td')[0].text) # just to see does it work or not

M2A



Let's extract the data and store in list so that it can easily be then appended in dataframes.

In [22]:
postalcode=[]
borough=[]
neighborhood=[]
for row in rows:
    postalcode.append(row.findAll('td')[0].text)
    borough.append(row.findAll('td')[1].text)
    neighborhood.append(row.findAll('td')[2].text)
    

Converting those lists into dataframes

In [23]:
toronto_df=pd.DataFrame()
toronto_df['Postal Code']=postalcode
toronto_df['Borough']=borough
toronto_df['Neighborhood']=neighborhood
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M2A\n,Not assigned\n,\n
1,M3A\n,North York\n,Parkwoods\n
2,M4A\n,North York\n,Victoria Village\n
3,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
4,M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n


We did using both method but it can easily be observed that using pandas.read_html method was much easy as well as it gave clean and proper table.</br>
We will use the dataframe obtained using pandas method.

# Cleaning and preprocessing the table

# 2. Drop the rows where Borough is not assigned.

In [24]:
toronto.head()


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Removing all the rows where borough is not assigned

In [25]:
toronto.drop(toronto[toronto['Borough']=='Not assigned'].index,inplace=True) 

In [26]:
toronto.head() #no more rows with not assigned values in borough

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [27]:
toronto.reset_index(inplace=True) #resting the index values

In [28]:
#Converting column names into more convenient to use
toronto.columns = toronto.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

# 3. Grouping the rows with same postal code

In [29]:
toronto_grouped=toronto.groupby(["postal_code","borough"],as_index=False).agg(lambda x:",".join(x))
toronto_grouped.head()

,postal_code,borough,neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 4. If neighborhood is not assigned then make it  same as borough

In [30]:
for r in toronto_grouped:
    if r[1]=='Not assigned':    #1 is the index for borough column
        r[1]=r[2]               #2 is the index of neighborhood column

In [31]:
toronto_grouped[toronto_grouped['neighborhood']=='NaN']

,postal_code,borough,neighborhood


No more any rows with not assigned neighborhood

# 5. Finally printing the shape of dataframe

In [32]:
toronto_grouped.shape

(103, 3)

# Part - 2 of Assignment

# 6. Merging lattitute and longitude columns from other datasets

We need to get lattitudes and longitudes of the corresponding postal codes of toronto.

In [54]:
lat_lang=pd.read_csv("https://cocl.us/Geospatial_data")
lat_lang.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
# toronto_merged=pd.merge(toronto_grouped,lat_lang,on='postal_code')
lat_lang.columns=lat_lang.columns.str.strip().str.lower().str.replace(' ','_')
lat_lang.columns

Index(['postal_code', 'latitude', 'longitude'], dtype='object')

In [56]:
toronto_merged=pd.merge(toronto_grouped,lat_lang,on='postal_code')

In [57]:
toronto_merged.head()

,postal_code,borough,neighborhood,latitude,longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Hoila it's done!!!!!

# Part-3 of  Assignment

<h1> Segmenting and clustering neighbourhood of Toronto</h1>

Let's visualize toronto using folium plotting the borough and neighborhoof in toronto.

In [61]:
# !conda install -c conda-forge geopy --yes #installing folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [63]:
# !conda install -c conda-forge folium --yes 
!pip install folium
import folium # map rendering library

     |████████████████████████████████| 92kB 6.7MB/s eta 0:00:011


# 7. Getting latitude and longitude of toronto using geopy

In [64]:
address = 'Toronto'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# 8. Creating map of toronto with neighborhoods using folium

In [69]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['borough'], toronto_merged['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto


# 9.Filtering only borough that has 'toronto' in it.

In [70]:
# filter borough names that contain the word Toronto
borough_names = list(toronto_merged.borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [74]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_merged[toronto_merged['borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,postal_code,borough,neighborhood,latitude,longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [76]:
# create map of Toronto using latitude and longitude values
map_toronto_new = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['latitude'], toronto_df_new['longitude'], toronto_df_new['borough'], toronto_df_new['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# 10.Using FourSquare to explore more.

In [102]:
# define Foursquare Credentials and Version
#it has been hidden for security purposes.
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


Let us get top venues around radius of 500 for new dataframes of borough that has Toronto in it

In [80]:
RADIUS=500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['latitude'], toronto_df_new['longitude'], toronto_df_new['postal_code'], toronto_df_new['borough'], toronto_df_new['neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        RADIUS, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [81]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['postal_code', 'borough', 'neighborhood', 'borough_latitude', 'borough_longitude', 'venue_name', 'venue_latitude', 'venue_longitude', 'venue_category']

print(venues_df.shape)
venues_df.head()

(1684, 9)


,postal_code,borough,neighborhood,borough_latitude,borough_longitude,venue_name,venue_latitude,venue_longitude,venue_category
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


let us check for the categories returned by foursquare.

In [83]:
print('There are {} uniques categories.'.format(len(venues_df['venue_category'].unique())))

There are 232 uniques categories.


# 11.Analyzing the venues

In [84]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['venue_category']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['postal_code'] = venues_df['postal_code'] 
toronto_onehot['borough'] = venues_df['borough'] 
toronto_onehot['neighborhoods'] = venues_df['neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(1684, 235)


,postal_code,borough,neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


let's group these on neighborhood

In [85]:
toronto_grouped = toronto_onehot.groupby(["postal_code", "borough", "neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(39, 235)


,postal_code,borough,neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.20000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
1,M4K,East Toronto,The Danforth West / Riverdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.02381,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.02381
2,M4L,East Toronto,India Bazaar / The Beaches West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.02439,0.0,0.0,0.02439
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000


let's create a dataframe to display top 10 venues for each postal code

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['postal_code', 'borough', 'neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['postal_code'] = toronto_grouped['postal_code']
neighborhoods_venues_sorted['borough'] = toronto_grouped['borough']
neighborhoods_venues_sorted['neighborhoods'] = toronto_grouped['neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(39, 13)


,postal_code,borough,neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Neighborhood,Coffee Shop,Health Food Store,Pub,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run
1,M4K,East Toronto,The Danforth West / Riverdale,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Pub,Indian Restaurant,Diner
2,M4L,East Toronto,India Bazaar / The Beaches West,Park,Gym,Pub,Light Rail Station,Sandwich Place,Burrito Place,Restaurant,Fast Food Restaurant,Italian Restaurant,Steakhouse
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


# 12. Clustering toronto into 5 clusters

Running k-means to perform clustering

In [89]:
from  sklearn.cluster import KMeans

In [91]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["postal_code", "borough", "neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 4, 1, 1, 1, 3, 1], dtype=int32)

Making a new dataframe that contains the labels of cluster assigned as well as other details.

In [100]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_clustered = toronto_df_new.copy()

# add clustering labels
toronto_clustered["cluster_labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_clustered = toronto_clustered.join(neighborhoods_venues_sorted.drop(["borough", "neighborhoods"], 1).set_index("postal_code"), on="postal_code")

print(toronto_clustered.shape)
toronto_clustered.head() 

(39, 16)


,postal_code,borough,neighborhood,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Coffee Shop,Health Food Store,Pub,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Pub,Indian Restaurant,Diner
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,1,Park,Gym,Pub,Light Rail Station,Sandwich Place,Burrito Place,Restaurant,Fast Food Restaurant,Italian Restaurant,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


# 13. Visualizing the clusters

In [101]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_clustered['latitude'], toronto_clustered['longitude'], toronto_clustered['postal_code'], toronto_clustered['borough'], toronto_clustered['neighborhood'], toronto_clustered['cluster_labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

It is evident that most of them falls in only one cluster (the purple one)

In [103]:
toronto_clustered.loc[toronto_clustered['cluster_labels'] == 0, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Neighborhood,Coffee Shop,Health Food Store,Pub,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run


In [104]:
toronto_clustered.loc[toronto_clustered['cluster_labels'] == 1, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Pub,Indian Restaurant,Diner
2,East Toronto,1,Park,Gym,Pub,Light Rail Station,Sandwich Place,Burrito Place,Restaurant,Fast Food Restaurant,Italian Restaurant,Steakhouse
3,East Toronto,1,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
5,Central Toronto,1,Gym,Hotel,Park,Breakfast Spot,Food & Drink Shop,Sandwich Place,Department Store,Cosmetics Shop,Coworking Space,Colombian Restaurant
6,Central Toronto,1,Clothing Store,Coffee Shop,Spa,Seafood Restaurant,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Yoga Studio,Bagel Shop
7,Central Toronto,1,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Café,Sushi Restaurant,Gym,Coffee Shop,Gas Station,Diner
9,Central Toronto,1,Coffee Shop,Pub,Light Rail Station,American Restaurant,Sushi Restaurant,Bank,Fried Chicken Joint,Restaurant,Sports Bar,Pizza Place
11,Downtown Toronto,1,Coffee Shop,Café,Bakery,Pub,Italian Restaurant,Pizza Place,Pharmacy,Chinese Restaurant,Market,Restaurant
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Yoga Studio,Mediterranean Restaurant,Hotel,Café,Gastropub
13,Downtown Toronto,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Theater,Event Space


In [105]:
toronto_clustered.loc[toronto_clustered['cluster_labels'] == 2, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [106]:
toronto_clustered.loc[toronto_clustered['cluster_labels'] == 3, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Park,Summer Camp,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [107]:
toronto_clustered.loc[toronto_clustered['cluster_labels'] == 4, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Park,Swim School,Bus Line,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,Downtown Toronto,4,Park,Playground,Trail,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
23,Central Toronto,4,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


The cluser 1 seems to be most populated and popular where most popular places includes cafes and restaurant.If anybody wanted to start new business of sales ,this neighborhood could be a good place to start with.